# Environment setup

In [1]:
import os

import gromacs

NOTE: Some configuration directories are not set up yet: 
	/home/nikitos/.gromacswrapper
	/home/nikitos/.gromacswrapper/qscripts
	/home/nikitos/.gromacswrapper/templates
NOTE: You can create the configuration file and directories with:
	>>> import gromacs
	>>> gromacs.config.setup()


All settings are made in the cell below. 

* `SIMULATION_DIRECTORY` - The directory within which all simulations will be run and all files will be saved;
* `MDP_FILES_DIRECTORY` - Directory with parameter files (`.mdp`) for molecular dynamics simulations;
* `PDB_STRUCTURE` - PDB file with antibody structure. This file should be in `SIMULATION_DIRECTORY`.

In [2]:
# SIMULATION_DIRECTORY = 'remicade_humanization/remicade/'
SIMULATION_DIRECTORY = 'remicade_humanization/candidate_Aa/'
# SIMULATION_DIRECTORY = 'remicade_humanization/candidate_Cb/'
MDP_FILES_DIRECTORY = 'mdp_files/'

# PDB_STRUCTURE = '4g3y_Vdomens.pdb'
PDB_STRUCTURE = 'candidate_Aa_rank1_chothia_scheme.pdb'
# PDB_STRUCTURE = 'candidate_Cb_rank1_chothia_scheme.pdb'

# CDR residues
CDRS = {
    'H1': (26, 33),    # 26-32
    'H2': (52, 60),    # 52-59
    'H3': (101, 110),  # 101-109
    'L1': (144, 155),  # 24-34
    'L2': (170, 177),  # 50-56
    'L3': (209, 218),  # 89-97
} 

In [3]:
SIMULATION_DIRECTORY = os.path.abspath(SIMULATION_DIRECTORY)
MDP_FILES_DIRECTORY = os.path.abspath(MDP_FILES_DIRECTORY)

# Change the current working directory
os.chdir(SIMULATION_DIRECTORY)

# Print the current working directory
print(f'Current working directory: {os.getcwd()}')

Current working directory: /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Aa


# Function for trajectory processing

Based on [suggested workflow](https://manual.gromacs.org/current/user-guide/terminology.html#suggested-workflow) for `gmx trjconv`

In [4]:
def traj_processing(trajectory, 
                    structure, 
                    index='index.ndx', 
                    output_group='Protein', 
                    center=True, 
                    ur='compact', 
                    delete_initial_trj=True
                   ):
    dot_pos = trajectory.rfind('.')
    if dot_pos == -1:
        raise ValueError('Cannot determine file extension')
    
    trj_name = trajectory[:dot_pos]
    trj_ext = trajectory[dot_pos + 1:]
    
    gromacs.trjconv(f=trajectory,
                    s=structure,
                    n=index,
                    o=f'{trj_name}_whole.{trj_ext}',
                    pbc='whole',
                    input=(output_group)
                   )
    if delete_initial_trj:
        os.remove(trajectory)
    
    gromacs.trjconv(f=f'{trj_name}_whole.{trj_ext}',
                    s=structure,
                    n=index,
                    o=f'{trj_name}_whole_cluster.{trj_ext}',
                    pbc='cluster',
                    input=('Protein', output_group)
                   )
    os.remove(f'{trj_name}_whole.{trj_ext}')
    
    gromacs.trjconv(f=f'{trj_name}_whole_cluster.{trj_ext}',
                    n=index,
                    o=f'{trj_name}_noPBC.{trj_ext}',
                    pbc='nojump',
                    input=(output_group)
                   )
    os.remove(f'{trj_name}_whole_cluster.{trj_ext}')
    
    if center:
        gromacs.trjconv(f=f'{trj_name}_noPBC.{trj_ext}',
                        s=structure,
                        n=index,
                        o=f'{trj_name}_noPBC_center.{trj_ext}',
                        pbc='mol',
                        center=True,
                        ur=ur,
                        input=('Protein', output_group)
                       )
        os.remove(f'{trj_name}_noPBC.{trj_ext}')

# MD simulation

## Topology generation

Here we create files with topology (`.top`) and coordinates (`.gro`). Amber99sb force field is used, and tip3p for water model.

In [5]:
if not os.path.exists('structures/'):
    os.mkdir('structures')

gromacs.pdb2gmx(f=PDB_STRUCTURE,
                o='structures/processed.gro',
                p='topol.top',
                ff='amber99sb',
                water='tip3p'
                )

Using the Amber99sb force field in directory amber99sb.ff

going to rename amber99sb.ff/aminoacids.r2b

going to rename amber99sb.ff/dna.r2b

going to rename amber99sb.ff/rna.r2b
Reading candidate_Cb_rank1_chothia_scheme.pdb...
Read '', 1764 atoms

Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.

There are 2 chains and 0 blocks of water and 227 residues with 1764 atoms

  chain  #res #atoms

  1 'H'   120    934  

  2 'L'   107    830  

All occupancies are one

Reading residue database... (Amber99sb)

Processing chain 1 'H' (934 atoms, 120 residues)

Identified residue GLU1 as a starting terminus.

Identified residue SER113 as a ending terminus.

Checking for duplicate atoms....

Generating any missing hydrogen atoms and/or adding termini.

Now there are 120 residues with 1830 atoms
Chain time...

Making bonds...

Number of bonds was 1855, now 1854

Generating angles, dihedrals and pairs...

Making cmap torsions...

There are 4865 dihedrals,  3

                      :-) GROMACS - gmx pdb2gmx, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx pdb2gmx -f candidate_Cb_rank1_chothia_scheme.pdb -o structures/processed.gro -p topol.top -ff amber99sb -water tip3p

Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb.ff/aminoacids.r2b
Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb.ff/dna.r2b
Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb.ff/rna.r2b
All occupancies are one
Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb.ff/atomtypes.atp
Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb.ff/aminoacids.rtp
Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb.ff/dna.rtp
Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb.ff/r

(0, None, None)

## Defining unit cell and adding solvent

Let\`s define the box dimensions. We will use a rhombic dodecahedron box as the unit cell. After that we are filling the box with water

In [6]:
gromacs.editconf(f='structures/processed.gro',
                 c=True,
                 d=1.0,
                 bt='dodecahedron',
                 o='structures/newbox.gro'
                )

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3465 atoms
Volume: 115.674 nm^3, corresponds to roughly 52000 electrons
No velocities found
    system size :  5.325  4.133  5.255 (nm)
    diameter    :  5.788               (nm)
    center      :  0.930  0.119  0.420 (nm)
    box vectors :  5.325  4.134  5.255 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  : 115.67               (nm^3)
    shift       :  4.911  5.722  2.334 (nm)
new center      :  5.841  5.841  2.753 (nm)
new box vectors :  7.788  7.788  7.788 (nm)
new box angles  :  60.00  60.00  90.00 (degrees)
new box volume  : 334.01               (nm^3)


                      :-) GROMACS - gmx editconf, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx editconf -f structures/processed.gro -c -d 1.0 -bt dodecahedron -o structures/newbox.gro


GROMACS reminds you: "A robot will be truly autonomous when you instruct it to go to work and it decides to go to the beach instead." (Brad Templeton)



(0, None, None)

In [7]:
gromacs.solvate(cp='structures/newbox.gro',
                cs='spc216.gro',
                o='structures/solvate.gro',
                p='topol.top'
               )


         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. 68 (1964) pp. 441-451
-------- -------- --- Thank You --- -------- --------

Adding line for 9841 solvent molecules with resname (SOL) to topology file (topol.top)


                      :-) GROMACS - gmx solvate, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx solvate -cp structures/newbox.gro -cs spc216.gro -o structures/solvate.gro -p topol.top

Reading solute configuration
Reading solvent configuration

Initialising inter-atomic distances...
Generating solvent configuration
Will generate new solvent configuration of 5x5x3 boxes
Solvent box contains 36792 atoms in 12264 residues
Removed 4056 solvent atoms due to solvent-solvent overlap
Removed 3213 solvent atoms due to solute-solvent overlap
Sorting configuration
Found 1 molecule type:
    SOL (   3 atoms):  9841 residues
Generated solvent containing 29523 atoms in 9841 residues
Writing generated configuration to structures/solvate.gro

Output configuration contains 32988 atoms in 10068 residues
Volume                 :     334.00

(0, None, None)

## Adding ions

Since life does not exist at a net charge, we must add ions to our system.

In [8]:
gromacs.grompp(f=os.path.join(MDP_FILES_DIRECTORY, 'ions.mdp'),
               c='structures/solvate.gro',
               p='topol.top',
               o='ions.tpr'
              )

Setting the LD random seed to -337707021

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_H'

Excluding 3 bonded neighbours molecule type 'Protein_chain_L'

Excluding 2 bonded neighbours molecule type 'SOL'
Analysing residue names:
There are:   227    Protein residues
There are:  9841      Water residues
Analysing Protein...

This run will generate roughly 3 Mb of data


                       :-) GROMACS - gmx grompp, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx grompp -f /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/mdp_files/ions.mdp -c structures/solvate.gro -p topol.top -o ions.tpr

Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/mdp_files/ions.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Generating 1-4 interactions: fudge = 0.5

NOTE 2 [file topol.top, line 48]:
  System has non-zero total charge: 3.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.


(0, None, None)

In [9]:
gromacs.genion(s='ions.tpr',
               o='structures/solvate_ions.gro',
               p='topol.top',
               pname='NA',
               nname='CL',
               neutral=True,
               input='SOL'
              )

Will try to add 0 NA ions and 3 CL ions.
Select a continuous group of solvent molecules
Selected 13: 'SOL'

Processing topology
Replacing 3 solute molecules in topology file (topol.top)  by 0 NA and 3 CL ions.


                       :-) GROMACS - gmx genion, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx genion -s ions.tpr -o structures/solvate_ions.gro -p topol.top -pname NA -nname CL -neutral

Reading file ions.tpr, VERSION 2023 (single precision)
Reading file ions.tpr, VERSION 2023 (single precision)
Group     0 (         System) has 32988 elements
Group     1 (        Protein) has  3465 elements
Group     2 (      Protein-H) has  1766 elements
Group     3 (        C-alpha) has   227 elements
Group     4 (       Backbone) has   681 elements
Group     5 (      MainChain) has   910 elements
Group     6 (   MainChain+Cb) has  1117 elements
Group     7 (    MainChain+H) has  1132 elements
Group     8 (      SideChain) has  2333 elements
Group     9 (    SideChain-H) has   856 elements
Group    10 (    Prot-Masses) has  3465 ele

(0, None, None)

## Adding CDR coordinates to index

For the subsequent analysis of the trajectories, we need the coordinates of the C-alpha atoms of the CDR residues, so we first write in the index file which of the atoms are part of the CDRs.

In [5]:
input_for_make_ndx = []

for cdr_name in CDRS.keys():
    for ri in range(*CDRS[cdr_name]):
        input_for_make_ndx.append(f'ri {ri}')

input_for_make_ndx.append('splitch 1')
input_for_make_ndx.append('q')

In [11]:
gromacs.make_ndx(f='structures/solvate_ions.gro',
                 input=input_for_make_ndx
                )

Going to read 0 old index file(s)
Analysing residue names:
There are:   227    Protein residues
There are:  9838      Water residues
There are:     3        Ion residues
Analysing Protein...

  0 System              : 32982 atoms
  1 Protein             :  3465 atoms
  2 Protein-H           :  1766 atoms
  3 C-alpha             :   227 atoms
  4 Backbone            :   681 atoms
  5 MainChain           :   910 atoms
  6 MainChain+Cb        :  1117 atoms
  7 MainChain+H         :  1132 atoms
  8 SideChain           :  2333 atoms
  9 SideChain-H         :   856 atoms
 10 Prot-Masses         :  3465 atoms
 11 non-Protein         : 29517 atoms
 12 Water               : 29514 atoms
 13 SOL                 : 29514 atoms
 14 non-Water           :  3468 atoms
 15 Ion                 :     3 atoms
 16 Water_and_ions      : 29517 atoms

 nr : group      '!': not  'name' nr name   'splitch' nr    Enter: list groups
 'a': atom       '&': and  'del' nr         'splitres' nr   'l': list residues
 't

                      :-) GROMACS - gmx make_ndx, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx make_ndx -f structures/solvate_ions.gro


Reading structure file

GROMACS reminds you: "Garbage Collecting..." (GNU Emacs)



(0, None, None)

## Energy minimization

Here we perform energy minimization to ensure that the system has no steric clashes or inappropriate geometry. After that, one can check plot of potential energy of system during minimization. It can be found in `plots/` directory.

In [12]:
os.mkdir('em')

gromacs.grompp(f=os.path.join(MDP_FILES_DIRECTORY, 'em.mdp'),
               c='structures/solvate_ions.gro',
               p='topol.top',
               n='index.ndx',
               o='em/em.tpr'
              )

Setting the LD random seed to -1342177573

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_H'

Excluding 3 bonded neighbours molecule type 'Protein_chain_L'

Excluding 2 bonded neighbours molecule type 'SOL'

Excluding 1 bonded neighbours molecule type 'CL'

The largest distance between excluded atoms is 0.448 nm between atom 309 and 1491
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 72x72x72, spacing 0.108 0.108 0.108

Estimate for the relative computational load of the PME mesh part: 0.30

This run will generate roughly 3 Mb of data


                       :-) GROMACS - gmx grompp, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx grompp -f /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/mdp_files/em.mdp -c structures/solvate_ions.gro -p topol.top -n index.ndx -o em/em.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/mdp_files/em.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group rest is 69429.00
The integrator does not provide a ensemble temperature, there is no system ensemble temperature

There was 1 NOTE

GROMACS reminds you: 

(0, None, None)

In [13]:
gromacs.mdrun(deffnm='em/em')

                       :-) GROMACS - gmx mdrun, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx mdrun -deffnm em/em

Reading file em/em.tpr, VERSION 2023 (single precision)
1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 12 OpenMP threads 


Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+02
   Number of steps    =        10000

Energy minimization has stopped, but the forces have not converged to the
requested precision Fmax < 100 (which may not be possible for your system).
It stopped because the algorithm tried to make a new step whose size was too
small, or there was no change in the energy since last step. Eith

(0, None, None)

In [14]:
if not os.path.exists('analysis/'):
    os.mkdir('analysis')

gromacs.energy(f='em/em.edr',
               o='analysis/potential.xvg',
               input=('Potential', '0')
              )


Statistics over 6802 steps [ 0.0000 through 6801.0000 ps ], 1 data sets
All statistics are over 5377 points (frames)

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Potential                   -503104       5900    16788.7   -38691.1  (kJ/mol)


                       :-) GROMACS - gmx energy, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx energy -f em/em.edr -o analysis/potential.xvg

Opened em/em.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Coulomb-(SR)  
  9  Coul.-recip.    10  Potential       11  Pressure        12  Vir-XX        
 13  Vir-XY          14  Vir-XZ          15  Vir-YX          16  Vir-YY        
 17  Vir-YZ          18  Vir-ZX          19  Vir-ZY          20  Vir-ZZ        
 

(0, None, None)

## NVT equilibration

In [15]:
os.mkdir('nvt')

gromacs.grompp(f=os.path.join(MDP_FILES_DIRECTORY, 'nvt.mdp'),
               c='em/em.gro',
               p='topol.top',
               n='index.ndx',
               o='nvt/nvt.tpr'
              )

Setting the LD random seed to -730343748

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_H'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'Protein_chain_L'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'CL'

turning H bonds into constraints...

Setting gen_seed to -549122

Velocities were taken from a Maxwell distribution at 310 K

The largest distance between excluded atoms is 0.454 nm between atom 309 and 1491

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 310 K

Calculated rlist for 1x1 atom pair-list as 1.035 nm, buffer size 0.035 nm

Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setu

                       :-) GROMACS - gmx grompp, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx grompp -f /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/mdp_files/nvt.mdp -c em/em.gro -p topol.top -n index.ndx -o nvt/nvt.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'
Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group Protein is 8695.62
Number of degrees of freedom in T-Coupling group non-Protein is 59034.39

GROMACS reminds you: "Years of calculations and the stress, My science is waiting, nearly complete" (Midlake)



(0, None, None)

In [16]:
gromacs.mdrun(deffnm='nvt/nvt')

                       :-) GROMACS - gmx mdrun, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx mdrun -deffnm nvt/nvt

Reading file nvt/nvt.tpr, VERSION 2023 (single precision)
Changing nstlist from 10 to 100, rlist from 1 to 1.157

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI thread
Using 6 OpenMP threads 

starting mdrun 'Protein in water'
1000000 steps,   2000.0 ps.

Writing final coordinates.

               Core t (s)   Wall t (s)        (%)
       Time:     4240.364      706.729      600.0
                 (ns/day)    (hour/ns)
Performance:      244.507        0.098

GROMACS re

(0, None, None)

In [17]:
if not os.path.exists('analysis/'):
    os.mkdir('analysis')

gromacs.energy(f='nvt/nvt.edr',
               o='analysis/temperature.xvg',
               input=('Temperature', '0')
              )


Statistics over 1000001 steps [ 0.0000 through 2000.0000 ps ], 1 data sets
All statistics are over 10001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 309.978      0.031    2.07646   0.176612  (K)


                       :-) GROMACS - gmx energy, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx energy -f nvt/nvt.edr -o analysis/temperature.xvg

Opened nvt/nvt.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Disper.-corr. 
  9  Coulomb-(SR)    10  Coul.-recip.    11  Potential       12  Kinetic-En.   
 13  Total-Energy    14  Conserved-En.   15  Temperature     16  Pres.-DC      
 17  Pressure        18  Constr.-rmsd    19  Vir-XX          20  Vir-XY    

(0, None, None)

In [6]:
traj_processing('nvt/nvt.trr', 'nvt/nvt.tpr', output_group='System', delete_initial_trj=False)

                      :-) GROMACS - gmx trjconv, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Aa
Command line:
  gmx trjconv -f nvt/nvt.trr -s nvt/nvt.tpr -n index.ndx -o nvt/nvt_whole.trr -pbc whole

Will write trr: Trajectory in portable xdr format
Reading file nvt/nvt.tpr, VERSION 2023 (single precision)
Reading file nvt/nvt.tpr, VERSION 2023 (single precision)
Group     0 (         System) has 34227 elements
Group     1 (        Protein) has  3476 elements
Group     2 (      Protein-H) has  1773 elements
Group     3 (        C-alpha) has   227 elements
Group     4 (       Backbone) has   681 elements
Group     5 (      MainChain) has   910 elements
Group     6 (   MainChain+Cb) has  1117 elements
Group     7 (    MainChain+H) has  1133 elements
Group     8 (      SideChain) has  2343 elements
Group     9 (    SideChain-H) has   863 eleme

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'System'
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for clustering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 


Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Cluster


Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Cluster


Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Cluster

...


Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...


Last frame        200 time 2000.000    ->  frame    199 time 1990.000      
 ->  frame    200 time 2000.000      
Last written: frame    200 time 2000.000


GROMACS reminds you: "Cowardly refusing to create an empty archive" (GNU tar)

                      :-) GROMACS - gmx trjconv, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Aa
Command line:
  gmx trjconv -f nvt/nvt_noPBC.trr -s nvt/nvt.tpr -n index.ndx -o nvt/nvt_noPBC_center.trr -pbc mol -center -ur compact

Will write trr: Trajectory in portable xdr format
Reading file nvt/nvt.tpr, VERSION 2023 (single precision)
Reading file nvt/nvt.tpr, VERSION 2023 (single precision)
Group     0 (         System) has 34227 elements
Group     1 (        Protein) has  3476 elements
Group     2 (      Protein-H) has  1773 elements
Group     3 (        C-alpha) has   227 elements
Group     4 (       Bac

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'System'


Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'


Last frame        200 time 2000.000   
 ->  frame    200 time 2000.000      
Last written: frame    200 time 2000.000


GROMACS reminds you: "Our hands are tied by physics." (Christian Blau)



In [5]:
traj_processing('nvt/nvt.xtc', 'nvt/nvt.tpr', output_group='System')

                      :-) GROMACS - gmx trjconv, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx trjconv -f nvt/nvt.xtc -s nvt/nvt.tpr -n index.ndx -o nvt/nvt_whole.xtc -pbc whole


-------------------------------------------------------
Program:     gmx trjconv, version 2023
Source file: src/gromacs/commandline/cmdlineparser.cpp (line 271)
Function:    void gmx::CommandLineParser::parse(int*, char**)

Error in user input:
Invalid command-line options
  In command-line option -f
    File 'nvt/nvt.xtc' does not exist or is not accessible.
    The file could not be opened.
      Reason: No such file or directory
      (call to fopen() returned error code 2)

For more information and tips for troubleshooting, please check the GROMACS
website at http://www.gromacs.org/Documentation/Errors
-----------------------------------------

GromacsError: [Errno 1] Gromacs tool failed
Command invocation: printf "System\n" | gmx trjconv -f nvt/nvt.xtc -s nvt/nvt.tpr -n index.ndx -o nvt/nvt_whole.xtc -pbc whole

## NPT equilibration

In [21]:
os.mkdir('npt')

gromacs.grompp(f=os.path.join(MDP_FILES_DIRECTORY, 'npt.mdp'),
               c='nvt/nvt.gro',
               t='nvt/nvt.cpt',
               p='topol.top',
               n='index.ndx',
               o='npt/npt.tpr'
              )

Setting the LD random seed to -168855623

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_H'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'Protein_chain_L'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'CL'

turning H bonds into constraints...

The largest distance between excluded atoms is 0.447 nm between atom 309 and 1491

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 310 K

Calculated rlist for 1x1 atom pair-list as 1.035 nm, buffer size 0.035 nm

Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

Reading Coordinates, Velocities and Box size from old trajectory

Will read whole traj

                       :-) GROMACS - gmx grompp, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx grompp -f /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/mdp_files/npt.mdp -c nvt/nvt.gro -t nvt/nvt.cpt -p topol.top -n index.ndx -o npt/npt.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'
Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group Protein is 8695.62
Number of degrees of freedom in T-Coupling group non-Protein is 59034.39
Last frame         -1 time 2000.000   

GROMACS reminds you: "I'm Gonna Get Medieval On Your Ass" (Pulp Fiction)



(0, None, None)

In [22]:
gromacs.mdrun(deffnm='npt/npt')

                       :-) GROMACS - gmx mdrun, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx mdrun -deffnm npt/npt

Reading file npt/npt.tpr, VERSION 2023 (single precision)
Changing nstlist from 10 to 100, rlist from 1 to 1.157

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI thread
Using 6 OpenMP threads 

starting mdrun 'Protein in water'
1000000 steps,   2000.0 ps.

Writing final coordinates.

               Core t (s)   Wall t (s)        (%)
       Time:     4336.941      722.825      600.0
                 (ns/day)    (hour/ns)
Performance:      239.062        0.100

GROMACS re

(0, None, None)

In [23]:
if not os.path.exists('analysis/'):
    os.mkdir('analysis')

gromacs.energy(f='npt/npt.edr',
               o='analysis/pressure.xvg',
               input=('Pressure', '0')
              )

gromacs.energy(f='npt/npt.edr',
               o='analysis/density.xvg',
               input=('Density', '0')
              )


Statistics over 1000001 steps [ 0.0000 through 2000.0000 ps ], 1 data sets
All statistics are over 10001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Pressure                   -1.16963       0.74    191.944  -0.903926  (bar)

Statistics over 1000001 steps [ 0.0000 through 2000.0000 ps ], 1 data sets
All statistics are over 10001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Density                     1013.16       0.11    5.92155   0.287984  (kg/m^3)


                       :-) GROMACS - gmx energy, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx energy -f npt/npt.edr -o analysis/pressure.xvg

Opened npt/npt.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Disper.-corr. 
  9  Coulomb-(SR)    10  Coul.-recip.    11  Potential       12  Kinetic-En.   
 13  Total-Energy    14  Conserved-En.   15  Temperature     16  Pres.-DC      
 17  Pressure        18  Constr.-rmsd    19  Box-X           20  Box-Y        

(0, None, None)

In [7]:
traj_processing('npt/npt.trr', 'npt/npt.tpr', output_group='System', delete_initial_trj=False)

                      :-) GROMACS - gmx trjconv, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Aa
Command line:
  gmx trjconv -f npt/npt.trr -s npt/npt.tpr -n index.ndx -o npt/npt_whole.trr -pbc whole

Will write trr: Trajectory in portable xdr format
Reading file npt/npt.tpr, VERSION 2023 (single precision)
Reading file npt/npt.tpr, VERSION 2023 (single precision)
Group     0 (         System) has 34227 elements
Group     1 (        Protein) has  3476 elements
Group     2 (      Protein-H) has  1773 elements
Group     3 (        C-alpha) has   227 elements
Group     4 (       Backbone) has   681 elements
Group     5 (      MainChain) has   910 elements
Group     6 (   MainChain+Cb) has  1117 elements
Group     7 (    MainChain+H) has  1133 elements
Group     8 (      SideChain) has  2343 elements
Group     9 (    SideChain-H) has   863 eleme

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'System'
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for clustering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 


Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Cluster


Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Cluster


Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Cluster

...


Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...


Last frame        200 time 2000.000    ->  frame    199 time 1990.000      
 ->  frame    200 time 2000.000      
Last written: frame    200 time 2000.000


GROMACS reminds you: "Dr Pauling, how do you have so many good ideas? Well David, I have a lot of ideas and throw away the bad ones." (Linus Pauling)

                      :-) GROMACS - gmx trjconv, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Aa
Command line:
  gmx trjconv -f npt/npt_noPBC.trr -s npt/npt.tpr -n index.ndx -o npt/npt_noPBC_center.trr -pbc mol -center -ur compact

Will write trr: Trajectory in portable xdr format
Reading file npt/npt.tpr, VERSION 2023 (single precision)
Reading file npt/npt.tpr, VERSION 2023 (single precision)
Group     0 (         System) has 34227 elements
Group     1 (        Protein) has  3476 elements
Group     2 (      Protein-H) has  1773 elements


Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'System'


Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'


Last frame        200 time 2000.000   
 ->  frame    200 time 2000.000      
Last written: frame    200 time 2000.000


GROMACS reminds you: "Perl: The only language that looks the same before and after RSA encryption." (Keith Bostic)



In [25]:
traj_processing('npt/npt.xtc', 'npt/npt.tpr', output_group='System')

                      :-) GROMACS - gmx trjconv, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx trjconv -f npt/npt.xtc -s npt/npt.tpr -n index.ndx -o npt/npt_whole.xtc -pbc whole

Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file npt/npt.tpr, VERSION 2023 (single precision)
Reading file npt/npt.tpr, VERSION 2023 (single precision)
Group     0 (         System) has 32982 elements
Group     1 (        Protein) has  3465 elements
Group     2 (      Protein-H) has  1766 elements
Group     3 (        C-alpha) has   227 elements
Group     4 (       Backbone) has   681 elements
Group     5 (      MainChain) has   910 elements
Group     6 (   MainChain+Cb) has  1117 elements
Group     7 (    MainChain+H) has  1132 elements
Group     8 (      SideChain) has  2333 elements
Group     9 (    SideChain-H) 

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'System'
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for clustering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 


Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Cluster


Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Cluster


Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Cluster

...

Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clusteri

Last frame      10000 time 2000.000    ->  frame   9999 time 1999.800      
 ->  frame  10000 time 2000.000      
Last written: frame  10000 time 2000.000


GROMACS reminds you: "Your daddy ain't your daddy but your daddy don't know" (Dalahan)

                      :-) GROMACS - gmx trjconv, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx trjconv -f npt/npt_whole_cluster.xtc -n index.ndx -o npt/npt_noPBC.xtc -pbc nojump

Will write xtc: Compressed trajectory (portable xdr format): xtc
Group     0 (         System) has 32982 elements
Group     1 (        Protein) has  3465 elements
Group     2 (      Protein-H) has  1766 elements
Group     3 (        C-alpha) has   227 elements
Group     4 (       Backbone) has   681 elements
Group     5 (      MainChain) has   910 elements
Group     6 (   MainChain+Cb) has  1117 elements

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'System'


Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'


Last frame      10000 time 2000.000   
 ->  frame  10000 time 2000.000      
Last written: frame  10000 time 2000.000


GROMACS reminds you: "In ancient times they had no statistics so they had to fall back on lies." (Stephen Leacock)



## Production run

### Short test production run

In [26]:
os.mkdir('prod_test')

gromacs.grompp(f=os.path.join(MDP_FILES_DIRECTORY, 'production_short.mdp'),
               c='npt/npt.gro',
               t='npt/npt.cpt',
               p='topol.top',
               n='index.ndx',
               o='prod_test/prod_test.tpr'
              )

Setting the LD random seed to -28331573

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_H'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'Protein_chain_L'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'CL'

turning H bonds into constraints...

The largest distance between excluded atoms is 0.439 nm between atom 309 and 1491

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 310 K

Calculated rlist for 1x1 atom pair-list as 1.036 nm, buffer size 0.036 nm

Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

Reading Coordinates, Velocities and Box size from old trajectory

Will read whole traje

                       :-) GROMACS - gmx grompp, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx grompp -f /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/mdp_files/production_short.mdp -c npt/npt.gro -t npt/npt.cpt -p topol.top -n index.ndx -o prod_test/prod_test.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'
Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group Protein is 8695.62
Number of degrees of freedom in T-Coupling group non-Protein is 59034.39
Last frame         -1 time 2000.000   

GROMACS reminds you: "You Leave Me Dry" (P.J. Harvey)



(0, None, None)

In [27]:
gromacs.mdrun(deffnm='prod_test/prod_test')

                       :-) GROMACS - gmx mdrun, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Cb
Command line:
  gmx mdrun -deffnm prod_test/prod_test

Reading file prod_test/prod_test.tpr, VERSION 2023 (single precision)
Changing nstlist from 10 to 100, rlist from 1 to 1.158

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI thread
Using 6 OpenMP threads 

starting mdrun 'Protein in water'
500000 steps,   1000.0 ps.

Writing final coordinates.

               Core t (s)   Wall t (s)        (%)
       Time:     2138.283      356.382      600.0
                 (ns/day)    (hour/ns)
Performance:      242.437  

(0, None, None)

In [8]:
traj_processing('prod_test/prod_test.xtc', 'prod_test/prod_test.tpr', delete_initial_trj=False)

                      :-) GROMACS - gmx trjconv, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Aa
Command line:
  gmx trjconv -f prod_test/prod_test.xtc -s prod_test/prod_test.tpr -n index.ndx -o prod_test/prod_test_whole.xtc -pbc whole

Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file prod_test/prod_test.tpr, VERSION 2023 (single precision)
Reading file prod_test/prod_test.tpr, VERSION 2023 (single precision)
Group     0 (         System) has 34227 elements
Group     1 (        Protein) has  3476 elements
Group     2 (      Protein-H) has  1773 elements
Group     3 (        C-alpha) has   227 elements
Group     4 (       Backbone) has   681 elements
Group     5 (      MainChain) has   910 elements
Group     6 (   MainChain+Cb) has  1117 elements
Group     7 (    MainChain+H) has  1133 elements
Group     8 (      

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 1: 'Protein'
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for clustering
Selected 1: 'Protein'
Select group for output
Selected 1: 'Protein'
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 

Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clusteri


Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Cluster


Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Cluster

...

Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clustering iteration 2 of 2...
Clusteri

Last frame       5000 time 1000.000    ->  frame   4999 time  999.800      
 ->  frame   5000 time 1000.000      
Last written: frame   5000 time 1000.000


GROMACS reminds you: "You're like them scientists on TV explaining black holes. More you talk, less I get" (Jess Walter)

                      :-) GROMACS - gmx trjconv, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Aa
Command line:
  gmx trjconv -f prod_test/prod_test_whole_cluster.xtc -n index.ndx -o prod_test/prod_test_noPBC.xtc -pbc nojump

Will write xtc: Compressed trajectory (portable xdr format): xtc
Group     0 (         System) has 34227 elements
Group     1 (        Protein) has  3476 elements
Group     2 (      Protein-H) has  1773 elements
Group     3 (        C-alpha) has   227 elements
Group     4 (       Backbone) has   681 elements
Group     5 (      MainChain) has   910

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 1: 'Protein'


Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 1: 'Protein'


Last frame       5000 time 1000.000   
 ->  frame   5000 time 1000.000      
Last written: frame   5000 time 1000.000


GROMACS reminds you: "A robot will be truly autonomous when you instruct it to go to work and it decides to go to the beach instead." (Brad Templeton)



### Main production run

In [54]:
os.mkdir('prod')

gromacs.grompp(f=os.path.join(MDP_FILES_DIRECTORY, 'production.mdp'),
               c='npt/npt.gro',
               t='npt/npt.cpt',
               p='topol.top',
               n='index.ndx',
               o='prod/prod.tpr'
              )

In [57]:
gromacs.mdrun(deffnm='prod/prod')

                       :-) GROMACS - gmx mdrun, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Aa
Command line:
  gmx mdrun -deffnm prod/prod

Reading file prod/prod.tpr, VERSION 2023 (single precision)
Changing nstlist from 10 to 100, rlist from 1 to 1.157

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI thread
Using 6 OpenMP threads 



starting mdrun 'Protein in water'
40000000 steps,  80000.0 ps.

Writing final coordinates.

               Core t (s)   Wall t (s)        (%)
       Time:   183486.274    30581.047      600.0
                         8h29:41
                 (ns/day)    (hour/ns)
Performanc

(0, None, None)

In [58]:
gromacs.trjconv(s='prod/prod.tpr',
                f='prod/prod.xtc',
                n='index.ndx',
                o='prod/prod_noPBC.xtc',
                pbc='mol',
                center=True,
                ur='compact',
                input=('Protein_chain1', 'Protein')
               )

                      :-) GROMACS - gmx trjconv, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Aa
Command line:
  gmx trjconv -s prod/prod.tpr -f prod/prod.xtc -n index.ndx -o prod/prod_noPBC.xtc -pbc mol -center

Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file prod/prod.tpr, VERSION 2023 (single precision)
Reading file prod/prod.tpr, VERSION 2023 (single precision)
Group     0 (         System) has 34227 elements
Group     1 (        Protein) has  3476 elements
Group     2 (      Protein-H) has  1773 elements
Group     3 (        C-alpha) has   227 elements
Group     4 (       Backbone) has   681 elements
Group     5 (      MainChain) has   910 elements
Group     6 (   MainChain+Cb) has  1117 elements
Group     7 (    MainChain+H) has  1133 elements
Group     8 (      SideChain) has  2343 elements
Group     9 ( 

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'


Last frame      40000 time 80000.000   
 ->  frame  40000 time 80000.000      
Last written: frame  40000 time 80000.000


GROMACS reminds you: "How wonderful that we have met with a paradox. Now we have some hope of making progress." (Niels Bohr)



(0, None, None)

# SASA calculation

In [9]:
if not os.path.exists('analysis/'):
    os.mkdir('analysis')

gromacs.sasa(f='prod_test/prod_test_noPBC_center.xtc',
             s='prod_test/prod_test.tpr',
             n='index.ndx',
             o='analysis/sasa.xvg',
             fgroup='Protein',
             surface='Protein',
             output=[f'"r_{gr[3:]}"' for gr in input_for_make_ndx[:-2]]
            )

                        :-) GROMACS - gmx sasa, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Aa
Command line:
  gmx sasa -f prod_test/prod_test_noPBC_center.xtc -s prod_test/prod_test.tpr -n index.ndx -o analysis/sasa.xvg -fgroup Protein -surface Protein -output "r_26" "r_27" "r_28" "r_29" "r_30" "r_31" "r_32" "r_52" "r_53" "r_54" "r_55" "r_56" "r_57" "r_58" "r_59" "r_101" "r_102" "r_103" "r_104" "r_105" "r_106" "r_107" "r_108" "r_109" "r_144" "r_145" "r_146" "r_147" "r_148" "r_149" "r_150" "r_151" "r_152" "r_153" "r_154" "r_170" "r_171" "r_172" "r_173" "r_174" "r_175" "r_176" "r_209" "r_210" "r_211" "r_212" "r_213" "r_214" "r_215" "r_216" "r_217"

Reading file prod_test/prod_test.tpr, VERSION 2023 (single precision)
Reading file prod_test/prod_test.tpr, VERSION 2023 (single precision)

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++



         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx sasa uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. 68 (1964) pp. 441-451
-------- -------- --- Thank You --- -------- --------



Last frame       5000 time 1000.000   
Analyzed 5001 frames, last time 1000.000

GROMACS reminds you: "We cannot wait for Nature's good graces - to take them from her is our goal" (Ivan Michurin)



(0, None, None)

# Movability calculation

In [10]:
if not os.path.exists('analysis/'):
    os.mkdir('analysis')

gromacs.rmsf(f='prod_test/prod_test_noPBC_center.xtc',
             s='em/em.gro',
             o='analysis/rmsf.xvg',
             od='analysis/rmsdev.xvg',
             res=True,
             input=('C-alpha')
            )

                        :-) GROMACS - gmx rmsf, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Aa
Command line:
  gmx rmsf -f prod_test/prod_test_noPBC_center.xtc -s em/em.gro -o analysis/rmsf.xvg -od analysis/rmsdef.xvg -res

Select group(s) for root mean square calculation
Group     0 (         System) has 34227 elements
Group     1 (        Protein) has  3476 elements
Group     2 (      Protein-H) has  1773 elements
Group     3 (        C-alpha) has   227 elements
Group     4 (       Backbone) has   681 elements
Group     5 (      MainChain) has   910 elements
Group     6 (   MainChain+Cb) has  1117 elements
Group     7 (    MainChain+H) has  1133 elements
Group     8 (      SideChain) has  2343 elements
Group     9 (    SideChain-H) has   863 elements
Group    10 (    Prot-Masses) has  3476 elements
Group    11 (    non-Protein) has 30751 


         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

Selected 3: 'C-alpha'


Last frame       5000 time 1000.000   

GROMACS reminds you: "Sincerity is the key to success. Once you can fake that you've got it made." (Groucho Marx)



(0, None, None)